In [25]:
import pandas as pd
import ta 
import optuna 
from itertools import combinations, chain 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [26]:
def powerset(s):
    return chain.from_iterable(combinations(s,r) for r in range(1,len(s)+1))

In [27]:
def file_features(data, ds_type: str):
    data1=pd.DataFrame()
    #Calcular indicadores tecnicos
    cmf_data = ta.volume.ChaikinMoneyFlowIndicator(data.High, data.Low, data.Close, data.Volume, window = 14)
    rsi_data = ta.momentum.RSIIndicator(data.Close, window=14)
    
    data1["CMF"] = cmf_data.chaikin_money_flow()
    data1["RSI"] = rsi_data.rsi()
    # Calcular la volatilidad
    data1['Volatility'] = data['High'] - data['Low']
    data1['Close_Lag0'] = data['Close']
    # Calcular las tendencias
    for i in range(1, 5 + 1):
        data1[f'Close_Lag{i}'] = data['Close'].shift(i)
    #Variable ded respuesta
    if ds_type == "buy":
        data1['Response'] = (data['Close'] < data['Close'].shift(-10))
    else:
        data1['Response'] = (data['Close'] > data['Close'].shift(-10))
    
    data1 = data1.drop(data1.index[:30])
    data1 = data1.drop(data1.index[-30:])
    data1.reset_index(drop=True, inplace=True)
    
    return data1

In [28]:
def objective_log_regresor(trial, data):
    # Dividir los datos en conjuntos de entrenamiento y prueba
    X = data.iloc[:, :-1]
    # Selecciona la variable objetivo
    y = data.iloc[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # Definir los parámetros a optimizar
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    C = trial.suggest_loguniform('C', 0.001, 1000)
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])

    # Crear el modelo de regresión logística con los parámetros sugeridos
    model = LogisticRegression(penalty=penalty, C=C, solver=solver, max_iter=10_000, random_state=123)
    # Entrenar el modelo
    model.fit(X_train, y_train)
    # Calcular la precisión en el conjunto de prueba
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [29]:
def objective_svm(trial, data):
    # Dividir los datos en conjuntos de entrenamiento y prueba
    X = data.iloc[:, :-1]
    # Selecciona la variable objetivo
    y = data.iloc[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Definir los parámetros a optimizar
    C = trial.suggest_loguniform('C', 0.001, 1000)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    if kernel == 'poly':
        degree = trial.suggest_int('degree', 2, 5)
    else:
        degree = 3  # Valor predeterminado si el kernel no es 'poly'
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto']) if kernel in ['rbf', 'poly', 'sigmoid'] else 'scale'
    # Crear el modelo SVM con los parámetros sugeridos
    model = SVC(C=C, kernel=kernel, degree=degree, gamma=gamma, max_iter=10_000, random_state=123)
    # Entrenar el modelo
    model.fit(X_train, y_train)
    # Calcular la precisión en el conjunto de prueba
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [30]:
def objective_xgboost(trial, data):
    data = data.copy()
    # Dividir los datos en conjuntos de entrenamiento y prueba
    X = data.iloc[:, :-1]
    # Selecciona la variable objetivo
    y = data.iloc[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    # Definir los parámetros a optimizar
    n_estimators = trial.suggest_int('n_estimators', 100, 1000, step=100)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.5)
    subsample = trial.suggest_discrete_uniform('subsample', 0.5, 1.0, 0.1)
    colsample_bytree = trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1.0, 0.1)
    # Crear el modelo XGBoost con los parámetros sugeridos
    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        random_state=123,
        max_iter=10_000
    )
    # Entrenar el modelo
    model.fit(X_train, y_train)
    # Calcular la precisión en el conjunto de prueba
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [31]:
def optimize_params_log_regresor(data):
    # Crear un estudio Optuna para la optimización
    study = optuna.create_study(direction='maximize')
    
    # Función objetivo con el dataset como parámetro fijo
    objective_fn = lambda trial: objective_log_regresor(trial, data)
    
    # Ejecutar la optimización
    study.optimize(objective_fn, n_trials=20)

    # Obtener los mejores parámetros
    best_params = study.best_params
    best_accuracy = study.best_value

    return best_params, best_accuracy

In [32]:
def optimize_params_svm(data):
    # Crear un estudio Optuna para la optimización
    study = optuna.create_study(direction='maximize')
    
    # Función objetivo con el dataset como parámetro fijo
    objective_fn = lambda trial: objective_svm(trial, data)
    
    # Ejecutar la optimización
    study.optimize(objective_fn, n_trials=20)

    # Obtener los mejores parámetros
    best_params = study.best_params
    best_accuracy = study.best_value

    return best_params, best_accuracy

In [33]:
def optimize_params_xgboost(data):
    # Crear un estudio Optuna para la optimización
    study = optuna.create_study(direction='maximize')
    
    # Función objetivo con el dataset como parámetro fijo
    objective_fn = lambda trial: objective_xgboost(trial, data)
    
    # Ejecutar la optimización
    study.optimize(objective_fn, n_trials=20)

    # Obtener los mejores parámetros
    best_params = study.best_params
    best_accuracy = study.best_value

    return best_params, best_accuracy

In [34]:
def optimize_params(data):
    # Optimización de regresión logística
    best_params_lr, best_accuracy_lr = optimize_params_log_regresor(data)
    print("Mejores parámetros de regresión logística:", best_params_lr)
    # Optimización de SVM
    best_params_svm, best_accuracy_svm = optimize_params_svm(data)
    print("Mejores parámetros de SVM:", best_params_svm)
    # Optimización de XGBoost
    best_params_xgb, best_accuracy_xgb = optimize_params_xgboost(data)
    print("Mejores parámetros de XGBoost:", best_params_xgb)
    return best_params_svm, best_params_lr, best_params_xgb

In [35]:
def params(data: str):
    data_1d_train = pd.read_csv(data)
    data_1d_train = data_1d_train.dropna()
    dataresult_long_1d_train = file_features(data_1d_train, ds_type="buy")
    dataresult_short_1d_train = file_features(data_1d_train, ds_type="sell")
    best_params_svm_long, best_params_lr_long, best_params_xgb_long = optimize_params(dataresult_long_1d_train)
    best_params_svm_short, best_params_lr_short, best_params_xgb_short = optimize_params(dataresult_short_1d_train)
    
    return best_params_lr_long, best_params_svm_long, best_params_xgb_long, best_params_lr_short, best_params_svm_short, best_params_xgb_short

In [36]:
logistic_params_1d_long, svm_params_1d_long, xgboost_params_1d_long, logistic_params_1d_short, svm_params_1d_short, xgboost_params_1d_short = params("data/aapl_1d_train.csv")

[I 2024-04-09 22:32:37,383] A new study created in memory with name: no-name-6d2e4eb2-5f14-4133-8b40-c4a65089e954
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/583301789.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
[I 2024-04-09 22:32:41,582] Trial 0 finished with value: 0.6564593301435406 and parameters: {'penalty': 'l2', 'C': 0.1796765744160107, 'solver': 'saga'}. Best is trial 0 with value: 0.6564593301435406.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/583301789.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
[I 2024-04-0

Mejores parámetros de regresión logística: {'penalty': 'l1', 'C': 0.010968139046945382, 'solver': 'saga'}


/opt/anaconda3/envs/trading/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-04-09 22:33:11,160] Trial 0 finished with value: 0.41913875598086126 and parameters: {'C': 0.061414378859970746, 'kernel': 'linear'}. Best is trial 0 with value: 0.41913875598086126.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/721807434.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
/opt/anaconda3/envs/trading/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 202

Mejores parámetros de SVM: {'C': 0.43326595131065515, 'kernel': 'poly', 'degree': 2, 'gamma': 'scale'}


[I 2024-04-09 22:33:22,325] Trial 0 finished with value: 0.5492822966507177 and parameters: {'n_estimators': 600, 'max_depth': 5, 'learning_rate': 0.12433422150235085, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 0 with value: 0.5492822966507177.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/2369882222.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.5)
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/2369882222.py:12: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  subsample = trial.suggest_discrete_uniform('subsample', 0.5, 1.0, 0.1)
/var/folders/x

...

Mejores parámetros de regresión logística: {'penalty': 'l2', 'C': 0.6431546383805156, 'solver': 'liblinear'}


[I 2024-04-09 22:33:50,576] Trial 0 finished with value: 0.661244019138756 and parameters: {'C': 0.0035454674239493874, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 0 with value: 0.661244019138756.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/721807434.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
[I 2024-04-09 22:33:50,921] Trial 1 finished with value: 0.5014354066985646 and parameters: {'C': 0.39203213125879516, 'kernel': 'sigmoid', 'gamma': 'scale'}. Best is trial 0 with value: 0.661244019138756.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/721807434.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_

Mejores parámetros de SVM: {'C': 0.0035454674239493874, 'kernel': 'rbf', 'gamma': 'scale'}


[I 2024-04-09 22:34:00,554] Trial 0 finished with value: 0.5645933014354066 and parameters: {'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.06013607443118158, 'subsample': 0.9, 'colsample_bytree': 1.0}. Best is trial 0 with value: 0.5645933014354066.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/2369882222.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.5)
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/2369882222.py:12: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  subsample = trial.suggest_discrete_uniform('subsample', 0.5, 1.0, 0.1)
/var/folders/x

Mejores parámetros de XGBoost: {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.030090107781624707, 'subsample': 0.9, 'colsample_bytree': 0.8}


In [37]:
logistic_params_1h_long, svm_params_1h_long, xgboost_params_1h_long, logistic_params_1h_short, svm_params_1h_short, xgboost_params_1h_short = params("data/aapl_1h_train.csv")

[I 2024-04-09 22:34:11,627] A new study created in memory with name: no-name-88813fd5-bc2d-4508-bafc-4a535cd35406
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/583301789.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
[I 2024-04-09 22:34:11,927] Trial 0 finished with value: 0.3125 and parameters: {'penalty': 'l2', 'C': 5.2064407063007545, 'solver': 'saga'}. Best is trial 0 with value: 0.3125.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/583301789.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
[I 2024-04-09 22:34:12,152] Trial 1 

Mejores parámetros de regresión logística: {'penalty': 'l1', 'C': 0.0021323714279392757, 'solver': 'saga'}


/opt/anaconda3/envs/trading/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-04-09 22:34:15,083] Trial 15 finished with value: 0.671875 and parameters: {'C': 39.21074674382456, 'kernel': 'poly', 'degree': 4, 'gamma': 'auto'}. Best is trial 2 with value: 0.703125.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/721807434.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
/opt/anaconda3/envs/trading/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I

Mejores parámetros de SVM: {'C': 0.04584664364591048, 'kernel': 'poly', 'degree': 2, 'gamma': 'auto'}


[I 2024-04-09 22:34:15,444] Trial 0 finished with value: 0.375 and parameters: {'n_estimators': 500, 'max_depth': 9, 'learning_rate': 0.09546149786469105, 'subsample': 0.7, 'colsample_bytree': 0.8}. Best is trial 0 with value: 0.375.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/2369882222.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.5)
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/2369882222.py:12: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  subsample = trial.suggest_discrete_uniform('subsample', 0.5, 1.0, 0.1)
/var/folders/x7/frl5my251vx198wg4w_ndzxh

...

Mejores parámetros de regresión logística: {'penalty': 'l2', 'C': 0.0028315415752095535, 'solver': 'saga'}


[I 2024-04-09 22:34:24,412] Trial 17 finished with value: 0.21875 and parameters: {'C': 40.06753069465152, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 1 with value: 0.78125.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/721807434.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
/opt/anaconda3/envs/trading/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-04-09 22:34:24,425] Trial 18 finished with value: 0.21875 and parameters: {'C': 0.18274359857920905, 'kernel': 'linear'}. Best is trial 1 with value: 0.78125.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/721807434.py:9: 

Mejores parámetros de SVM: {'C': 0.2018232497206618, 'kernel': 'rbf', 'gamma': 'auto'}


[I 2024-04-09 22:34:25,099] Trial 1 finished with value: 0.375 and parameters: {'n_estimators': 1000, 'max_depth': 5, 'learning_rate': 0.018001348182327223, 'subsample': 1.0, 'colsample_bytree': 0.9}. Best is trial 0 with value: 0.390625.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/2369882222.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.5)
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/2369882222.py:12: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  subsample = trial.suggest_discrete_uniform('subsample', 0.5, 1.0, 0.1)
/var/folders/x7/frl5my251vx198wg4w_

Mejores parámetros de XGBoost: {'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.102602018285415, 'subsample': 0.5, 'colsample_bytree': 0.6}


In [38]:
logistic_params_1m_long, svm_params_1m_long, xgboost_params_1m_long, logistic_params_1m_short, svm_params_1m_short, xgboost_params_1m_short = params("data/aapl_1m_train.csv")

[I 2024-04-09 22:34:29,239] A new study created in memory with name: no-name-8caf7def-6613-4832-95ab-fe191fb893fc
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/583301789.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
[I 2024-04-09 22:34:29,245] Trial 0 finished with value: 0.328125 and parameters: {'penalty': 'l2', 'C': 2.8254439628942936, 'solver': 'liblinear'}. Best is trial 0 with value: 0.328125.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/583301789.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
[I 2024-04-09 22:34:29,252]

Mejores parámetros de regresión logística: {'penalty': 'l2', 'C': 0.0015827990076342706, 'solver': 'saga'}


/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/721807434.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
[I 2024-04-09 22:34:32,249] Trial 19 finished with value: 0.234375 and parameters: {'C': 0.33148860835040184, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 5 with value: 0.6875.
[I 2024-04-09 22:34:32,250] A new study created in memory with name: no-name-4c3e589e-161a-4541-9ef7-8f1228d8a9cd
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/2369882222.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.5)
/var/fo

Mejores parámetros de SVM: {'C': 208.31110635195978, 'kernel': 'rbf', 'gamma': 'auto'}


[I 2024-04-09 22:34:32,590] Trial 1 finished with value: 0.359375 and parameters: {'n_estimators': 600, 'max_depth': 10, 'learning_rate': 0.1895823003911276, 'subsample': 0.8, 'colsample_bytree': 1.0}. Best is trial 1 with value: 0.359375.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/2369882222.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.5)
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/2369882222.py:12: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  subsample = trial.suggest_discrete_uniform('subsample', 0.5, 1.0, 0.1)
/var/folders/x7/frl5my251vx198wg4w

...

Mejores parámetros de regresión logística: {'penalty': 'l2', 'C': 0.002105767490569824, 'solver': 'saga'}


/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/721807434.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
[I 2024-04-09 22:34:40,168] Trial 14 finished with value: 0.5 and parameters: {'C': 82.70898411262169, 'kernel': 'sigmoid', 'gamma': 'scale'}. Best is trial 5 with value: 0.78125.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/721807434.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
/opt/anaconda3/envs/trading/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Cons

Mejores parámetros de SVM: {'C': 89.33147882881373, 'kernel': 'linear'}


[I 2024-04-09 22:34:40,587] Trial 0 finished with value: 0.3125 and parameters: {'n_estimators': 900, 'max_depth': 9, 'learning_rate': 0.18076739947479364, 'subsample': 1.0, 'colsample_bytree': 1.0}. Best is trial 0 with value: 0.3125.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/2369882222.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.5)
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/2369882222.py:12: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  subsample = trial.suggest_discrete_uniform('subsample', 0.5, 1.0, 0.1)
/var/folders/x7/frl5my251vx198wg4w_ndz

Mejores parámetros de XGBoost: {'n_estimators': 400, 'max_depth': 3, 'learning_rate': 0.061659645644687434, 'subsample': 0.8, 'colsample_bytree': 0.7}


In [39]:
logistic_params_5m_long, svm_params_5m_long, xgboost_params_5m_long, logistic_params_5m_short, svm_params_5m_short, xgboost_params_5m_short = params("data/aapl_5m_train.csv")

[I 2024-04-09 22:34:44,582] A new study created in memory with name: no-name-f9ba9170-031e-463c-bbb1-ea6c39e84c3f
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/583301789.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
[I 2024-04-09 22:34:46,311] Trial 0 finished with value: 0.5003836317135549 and parameters: {'penalty': 'l2', 'C': 0.0018530287394076777, 'solver': 'saga'}. Best is trial 0 with value: 0.5003836317135549.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/583301789.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
[I 2024-0

Mejores parámetros de regresión logística: {'penalty': 'l2', 'C': 0.002395829495054093, 'solver': 'liblinear'}


/opt/anaconda3/envs/trading/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-04-09 22:41:17,069] Trial 0 finished with value: 0.49923273657289 and parameters: {'C': 0.07823691840138479, 'kernel': 'linear'}. Best is trial 0 with value: 0.49923273657289.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/721807434.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
/opt/anaconda3/envs/trading/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-04-09

Mejores parámetros de SVM: {'C': 0.002176996974479978, 'kernel': 'sigmoid', 'gamma': 'scale'}


[I 2024-04-09 22:47:20,636] Trial 0 finished with value: 0.49373401534526856 and parameters: {'n_estimators': 200, 'max_depth': 4, 'learning_rate': 0.03943694697221585, 'subsample': 0.6, 'colsample_bytree': 0.9}. Best is trial 0 with value: 0.49373401534526856.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/2369882222.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.5)
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/2369882222.py:12: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  subsample = trial.suggest_discrete_uniform('subsample', 0.5, 1.0, 0.1)
/var/folders

...

Mejores parámetros de regresión logística: {'penalty': 'l1', 'C': 0.0030591725482995644, 'solver': 'saga'}


/opt/anaconda3/envs/trading/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-04-09 22:59:19,624] Trial 0 finished with value: 0.49808184143222506 and parameters: {'C': 1.1232132644344264, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 0 with value: 0.49808184143222506.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/721807434.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
/opt/anaconda3/envs/trading/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warni

Mejores parámetros de SVM: {'C': 0.6739747156890435, 'kernel': 'linear'}


[I 2024-04-09 23:04:10,174] Trial 0 finished with value: 0.5019181585677749 and parameters: {'n_estimators': 300, 'max_depth': 8, 'learning_rate': 0.01886513820583103, 'subsample': 1.0, 'colsample_bytree': 0.8}. Best is trial 0 with value: 0.5019181585677749.
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/2369882222.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.5)
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_31732/2369882222.py:12: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  subsample = trial.suggest_discrete_uniform('subsample', 0.5, 1.0, 0.1)
/var/folders/x

Mejores parámetros de XGBoost: {'n_estimators': 900, 'max_depth': 9, 'learning_rate': 0.08242884326336312, 'subsample': 0.5, 'colsample_bytree': 1.0}


In [40]:
def buy_signals(data, logistic_params, svm_params, xgboost_params):
    buy_signals = pd.DataFrame()
    # Selecciona las características
    X = data.iloc[:, :-1]
    # Selecciona la variable objetivo
    y = data.iloc[:, -1]

    # Crear modelos con los parámetros proporcionados
    logistic_model = LogisticRegression(**logistic_params)
    svm_model = SVC(**svm_params)
    xgboost_model = XGBClassifier(**xgboost_params)

    # Entrenar los modelos con todo el conjunto de datos original
    logistic_model.fit(X, y)
    svm_model.fit(X, y)
    xgboost_model.fit(X, y)

    # Realizar predicciones en el conjunto de datos original
    predictions_lr = logistic_model.predict(X)
    predictions_svm = svm_model.predict(X)
    predictions_xgboost = xgboost_model.predict(X)
    predictions_xgboost_bool = predictions_xgboost.astype(bool)

    # Agregar las predicciones como nuevas columnas al conjunto de datos original
    buy_signals['predicciones_lr'] = predictions_lr
    buy_signals['predicciones_svm'] = predictions_svm
    buy_signals['predicciones_xgboost'] = predictions_xgboost_bool

    return buy_signals

In [41]:
def sell_signals(data, logistic_params, svm_params, xgboost_params):
    sell_signals = pd.DataFrame()
    # Selecciona las características
    X = data.iloc[:, :-1]
    # Selecciona la variable objetivo
    y = data.iloc[:, -1]

    # Crear modelos con los parámetros proporcionados
    logistic_model = LogisticRegression(**logistic_params)
    svm_model = SVC(**svm_params)
    xgboost_model = XGBClassifier(**xgboost_params)

    # Entrenar los modelos con todo el conjunto de datos original
    logistic_model.fit(X, y)
    svm_model.fit(X, y)
    xgboost_model.fit(X, y)

    # Realizar predicciones en el conjunto de datos original
    predictions_lr = logistic_model.predict(X)
    predictions_svm = svm_model.predict(X)
    predictions_xgboost = xgboost_model.predict(X)
    predictions_xgboost_bool = predictions_xgboost.astype(bool)

    # Agregar las predicciones como nuevas columnas al conjunto de datos original
    sell_signals['predicciones_lr'] = predictions_lr
    sell_signals['predicciones_svm'] = predictions_svm
    sell_signals['predicciones_xgboost'] = predictions_xgboost_bool

    return sell_signals

In [42]:
def backtest(data, buy_signals, sell_signals, stop_loss, take_profit, n_shares):
            history = []
            active_operations = []
            cash = 1_000_000
            com = 1.25 / 100

            for i, row in data.iterrows():
                # close active operation
                active_op_temp = []
                for operation in active_operations:
                    if operation["stop_loss"] > row.Close:
                        cash += (row.Close * operation["n_shares"]) * (1 - com)
                    elif operation["take_profit"] < row.Close:
                        cash += (row.Close * operation["n_shares"]) * (1 - com)
                    else:
                        active_op_temp.append(operation)
                active_operations = active_op_temp

                # check if we have enough cash
                if cash < (row.Close * (1 + com)):
                    asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
                    portfolio_value = cash + asset_vals
                    continue

                # Apply buy signals
                if buy_signals.loc[i].any():
                    active_operations.append({
                        "bought": row.Close,
                        "n_shares": n_shares,
                        "stop_loss": row.Close * stop_loss,
                        "take_profit": row.Close * take_profit
                    })

                    cash -= row.Close * (1 + com) * n_shares

                # Apply sell signals
                if sell_signals.loc[i].any():
                    active_op_temp = []
                    for operation in active_operations:
                        if operation["take_profit"] < row.Close or operation["stop_loss"] > row.Close:
                            cash += (row.Close * operation["n_shares"]) * (1 - com)
                        else:
                            active_op_temp.append(operation)
                    active_operations = active_op_temp

                asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
                portfolio_value = cash + asset_vals

            return portfolio_value

In [43]:
def optimize(trial, strategy, data):
    portfolio_value = 0

    stop_loss = trial.suggest_float("stop_loss", 0.80, 0.90)
    take_profit = trial.suggest_float("take_profit", 1.01, 1.10)
    n_shares = trial.suggest_int("n_shares", 20, 50)

    strat_params = {}

    buy_signals = pd.DataFrame()
    sell_signals = pd.DataFrame()

    if "logistic" in strategy:
        buy_signals["logistic"] = global_buy_signals["predicciones_lr"]
        sell_signals["logistic"] = global_sell_signals["predicciones_lr"]
        
    if "svm" in strategy:
        buy_signals["svm"] = global_buy_signals["predicciones_svm"]
        sell_signals["svm"] = global_sell_signals["predicciones_svm"]
        
    if "xg" in strategy:
        buy_signals["xg"] = global_buy_signals["predicciones_xgboost"]
        sell_signals["xg"] = global_sell_signals["predicciones_xgboost"]
    
    return backtest(data, buy_signals, sell_signals, stop_loss, take_profit, n_shares)

In [44]:
def optimize_file(data):
    data = data.drop(data.index[:30])
    data = data.drop(data.index[-30:])
    data.reset_index(drop=True, inplace=True)    
    strategies = list(powerset(["logistic", "svm", "xg"]))
    best_strat = None
    best_val = -1
    best_params = None

    for strat in strategies:
        study = optuna.create_study(direction="maximize")
        study.optimize(lambda x: optimize(x, strat, data), n_trials=20)
        value = study.best_value
        print(f"Best value found so far: {value}")
        if value > best_val:
            best_val = value
            best_strat = strat
            best_params = study.best_params
            print(best_strat)
            print(best_params)

    return {"file": data,
            "strat": best_strat,
            "value": best_val,
            "params": best_params}

In [45]:
#Ahora con los datos de prueba

In [46]:
data_1d_test = pd.read_csv("data/aapl_1d_test.csv")
data_1d_test = data_1d_test.dropna()
dataresult_long_1d_test = file_features(data_1d_test, ds_type="buy")
dataresult_short_1d_test = file_features(data_1d_test, ds_type="sell")
global_buy_signals = buy_signals(dataresult_long_1d_test, logistic_params_1d_long, svm_params_1d_long, xgboost_params_1d_long)
global_sell_signals = sell_signals(dataresult_short_1d_test, logistic_params_1d_short, svm_params_1d_short, xgboost_params_1d_short)
file_1d_test = optimize_file(data_1d_test)

[I 2024-04-09 23:04:39,051] A new study created in memory with name: no-name-a387df3d-2119-4533-8ff2-de0ea7a2fb42
[I 2024-04-09 23:04:39,395] Trial 0 finished with value: 997203.7031249984 and parameters: {'stop_loss': 0.8087582004477056, 'take_profit': 1.0715543423297715, 'n_shares': 25}. Best is trial 0 with value: 997203.7031249984.
[I 2024-04-09 23:04:39,638] Trial 1 finished with value: 957335.954124999 and parameters: {'stop_loss': 0.8648438641036478, 'take_profit': 1.0610304490742015, 'n_shares': 33}. Best is trial 0 with value: 997203.7031249984.
[I 2024-04-09 23:04:39,904] Trial 2 finished with value: 962757.2200000015 and parameters: {'stop_loss': 0.8134633279478706, 'take_profit': 1.0510433693891676, 'n_shares': 40}. Best is trial 0 with value: 997203.7031249984.
[I 2024-04-09 23:04:40,283] Trial 3 finished with value: 1021629.2119999956 and parameters: {'stop_loss': 0.8144025856309808, 'take_profit': 1.099524372514011, 'n_shares': 22}. Best is trial 3 with value: 1021629.21

Best value found so far: 1029366.813000001
('logistic',)
{'stop_loss': 0.8005506575401419, 'take_profit': 1.0999214390424767, 'n_shares': 28}


[I 2024-04-09 23:04:45,039] Trial 1 finished with value: 950062.6580000005 and parameters: {'stop_loss': 0.8774378310733604, 'take_profit': 1.0463924299233485, 'n_shares': 31}. Best is trial 1 with value: 950062.6580000005.
[I 2024-04-09 23:04:45,167] Trial 2 finished with value: 943867.7625000018 and parameters: {'stop_loss': 0.8840843270764565, 'take_profit': 1.0212089146230532, 'n_shares': 25}. Best is trial 1 with value: 950062.6580000005.
[I 2024-04-09 23:04:45,316] Trial 3 finished with value: 961518.5587499996 and parameters: {'stop_loss': 0.8071339746964747, 'take_profit': 1.0225876594353336, 'n_shares': 29}. Best is trial 3 with value: 961518.5587499996.
[I 2024-04-09 23:04:45,591] Trial 4 finished with value: 1001821.4312500032 and parameters: {'stop_loss': 0.8669871997250641, 'take_profit': 1.0885202605906574, 'n_shares': 45}. Best is trial 4 with value: 1001821.4312500032.
[I 2024-04-09 23:04:45,851] Trial 5 finished with value: 993461.3187500017 and parameters: {'stop_loss

Best value found so far: 1029420.0218749983
('svm',)
{'stop_loss': 0.8009945784210862, 'take_profit': 1.087604037382484, 'n_shares': 25}


[I 2024-04-09 23:04:49,816] Trial 2 finished with value: 1035994.3252500029 and parameters: {'stop_loss': 0.876295098865582, 'take_profit': 1.0572032491421204, 'n_shares': 34}. Best is trial 2 with value: 1035994.3252500029.
[I 2024-04-09 23:04:49,889] Trial 3 finished with value: 988526.7062500017 and parameters: {'stop_loss': 0.8018866708600044, 'take_profit': 1.0100351961331593, 'n_shares': 25}. Best is trial 2 with value: 1035994.3252500029.
[I 2024-04-09 23:04:50,093] Trial 4 finished with value: 1106042.9 and parameters: {'stop_loss': 0.8782120012705377, 'take_profit': 1.0857939517032702, 'n_shares': 50}. Best is trial 4 with value: 1106042.9.
[I 2024-04-09 23:04:50,261] Trial 5 finished with value: 1037706.8374999997 and parameters: {'stop_loss': 0.8565148088119743, 'take_profit': 1.0651076130348207, 'n_shares': 25}. Best is trial 4 with value: 1106042.9.
[I 2024-04-09 23:04:50,354] Trial 6 finished with value: 1001012.698375 and parameters: {'stop_loss': 0.8999807040693121, 'ta

...

Best value found so far: 1043237.325625002


[I 2024-04-09 23:05:04,841] Trial 0 finished with value: 989480.7381250042 and parameters: {'stop_loss': 0.8764637519963632, 'take_profit': 1.0882928263362397, 'n_shares': 45}. Best is trial 0 with value: 989480.7381250042.
[I 2024-04-09 23:05:05,460] Trial 1 finished with value: 1019865.0402499968 and parameters: {'stop_loss': 0.831217443588928, 'take_profit': 1.0972183092337293, 'n_shares': 22}. Best is trial 1 with value: 1019865.0402499968.
[I 2024-04-09 23:05:05,641] Trial 2 finished with value: 954850.4284999996 and parameters: {'stop_loss': 0.8737304140987268, 'take_profit': 1.0258373505623042, 'n_shares': 22}. Best is trial 1 with value: 1019865.0402499968.
[I 2024-04-09 23:05:05,879] Trial 3 finished with value: 949822.5054999982 and parameters: {'stop_loss': 0.8917620605400515, 'take_profit': 1.065565599843179, 'n_shares': 42}. Best is trial 1 with value: 1019865.0402499968.
[I 2024-04-09 23:05:06,231] Trial 4 finished with value: 999206.0643749994 and parameters: {'stop_loss

Best value found so far: 1036182.8874999987


[I 2024-04-09 23:05:11,099] Trial 0 finished with value: 1000356.5452500045 and parameters: {'stop_loss': 0.8822229418680884, 'take_profit': 1.0985251861155723, 'n_shares': 34}. Best is trial 0 with value: 1000356.5452500045.
[I 2024-04-09 23:05:11,317] Trial 1 finished with value: 957196.0500000026 and parameters: {'stop_loss': 0.8250711650705123, 'take_profit': 1.0322221933502305, 'n_shares': 40}. Best is trial 0 with value: 1000356.5452500045.
[I 2024-04-09 23:05:11,687] Trial 2 finished with value: 1021102.4921249996 and parameters: {'stop_loss': 0.8629013434712464, 'take_profit': 1.0993832765988587, 'n_shares': 37}. Best is trial 2 with value: 1021102.4921249996.
[I 2024-04-09 23:05:11,965] Trial 3 finished with value: 984080.2885000003 and parameters: {'stop_loss': 0.8839321537461932, 'take_profit': 1.0727846669417387, 'n_shares': 21}. Best is trial 2 with value: 1021102.4921249996.
[I 2024-04-09 23:05:12,269] Trial 4 finished with value: 988251.3812500006 and parameters: {'stop_

Best value found so far: 1027341.3014999973


In [47]:
data_1m_test = pd.read_csv("data/aapl_1m_train.csv")
data_1m_test = data_1m_test.dropna()
dataresult_long_1m_test = file_features(data_1m_test, ds_type="buy")
dataresult_short_1m_test = file_features(data_1m_test, ds_type="sell")
global_buy_signals = buy_signals(dataresult_long_1m_test, logistic_params_1m_long, svm_params_1m_long, xgboost_params_1m_long)
global_sell_signals = sell_signals(dataresult_short_1m_test, logistic_params_1m_short, svm_params_1m_short, xgboost_params_1m_short)
file_1m_test = optimize_file(data_1m_test)

/opt/anaconda3/envs/trading/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/trading/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2024-04-09 23:05:17,889] A new study created in memory with name: no-name-469beee3-32f6-4058-af44-e7a24f6613ee
[I 2024-04-09 23:05:18,190] Trial 0 finished with value: 1054504.9759955036 and parameters: {'stop_loss': 0.8822119526073423, 'take_profit': 1.058864986531052, 'n_shares': 40}. Best is trial 0 with value: 1054504.9759955036.
[I 2024-04-09 23:05:18,489] Trial 1 finished with value: 1060288.9560896498 and parameters: {'stop_loss': 0.8787028049974225, 'take_profit': 1.070292945862413, 'n_shares': 36}. Best is trial 1 with value: 1060288.9560896498.
[I 2024-04-09 23:05:18,782] Trial 2 finished with va

Best value found so far: 1100231.5506074994
('logistic',)
{'stop_loss': 0.8190689853637289, 'take_profit': 1.0987097566356174, 'n_shares': 50}


[I 2024-04-09 23:05:23,518] Trial 2 finished with value: 1012014.640967799 and parameters: {'stop_loss': 0.8372811284310965, 'take_profit': 1.0337681591055026, 'n_shares': 48}. Best is trial 0 with value: 1063396.1421432504.
[I 2024-04-09 23:05:23,640] Trial 3 finished with value: 1032298.4453533888 and parameters: {'stop_loss': 0.8325622058697301, 'take_profit': 1.0552481202165327, 'n_shares': 49}. Best is trial 0 with value: 1063396.1421432504.
[I 2024-04-09 23:05:23,783] Trial 4 finished with value: 1037769.1886608734 and parameters: {'stop_loss': 0.8739820163240124, 'take_profit': 1.0647126278036494, 'n_shares': 45}. Best is trial 0 with value: 1063396.1421432504.
[I 2024-04-09 23:05:23,955] Trial 5 finished with value: 1026028.9715181999 and parameters: {'stop_loss': 0.8638166124450346, 'take_profit': 1.0839357450182845, 'n_shares': 24}. Best is trial 0 with value: 1063396.1421432504.
[I 2024-04-09 23:05:24,136] Trial 6 finished with value: 1055607.6893937744 and parameters: {'sto

Best value found so far: 1063396.1421432504


[I 2024-04-09 23:05:26,439] Trial 1 finished with value: 1013228.1356430992 and parameters: {'stop_loss': 0.8621272429117883, 'take_profit': 1.0525882172747496, 'n_shares': 21}. Best is trial 0 with value: 1024245.2094216998.
[I 2024-04-09 23:05:26,658] Trial 2 finished with value: 1066683.6842156248 and parameters: {'stop_loss': 0.8551892772718674, 'take_profit': 1.0965527968637032, 'n_shares': 50}. Best is trial 2 with value: 1066683.6842156248.
[I 2024-04-09 23:05:26,844] Trial 3 finished with value: 1032860.9895622361 and parameters: {'stop_loss': 0.8045615547442301, 'take_profit': 1.079926854838101, 'n_shares': 31}. Best is trial 2 with value: 1066683.6842156248.
[I 2024-04-09 23:05:27,056] Trial 4 finished with value: 1043489.2063504508 and parameters: {'stop_loss': 0.8285166082582477, 'take_profit': 1.0931401675564176, 'n_shares': 34}. Best is trial 2 with value: 1066683.6842156248.
[I 2024-04-09 23:05:27,209] Trial 5 finished with value: 1024401.7972046606 and parameters: {'sto

...

Best value found so far: 1101182.1528062494
('logistic', 'xg')
{'stop_loss': 0.8045410770677325, 'take_profit': 1.0999994091707832, 'n_shares': 50}


[I 2024-04-09 23:05:43,317] Trial 1 finished with value: 1026388.4578195009 and parameters: {'stop_loss': 0.8898168805229859, 'take_profit': 1.054471082427608, 'n_shares': 40}. Best is trial 0 with value: 1044188.7123812007.
[I 2024-04-09 23:05:43,545] Trial 2 finished with value: 1053721.9607858518 and parameters: {'stop_loss': 0.8755347201688687, 'take_profit': 1.0931514570236345, 'n_shares': 42}. Best is trial 2 with value: 1053721.9607858518.
[I 2024-04-09 23:05:43,593] Trial 3 finished with value: 994790.4223634115 and parameters: {'stop_loss': 0.8618144798507944, 'take_profit': 1.0114389861731343, 'n_shares': 33}. Best is trial 2 with value: 1053721.9607858518.
[I 2024-04-09 23:05:43,645] Trial 4 finished with value: 995923.6439581877 and parameters: {'stop_loss': 0.8017107474630357, 'take_profit': 1.01207437063522, 'n_shares': 31}. Best is trial 2 with value: 1053721.9607858518.
[I 2024-04-09 23:05:43,874] Trial 5 finished with value: 1057223.2489029989 and parameters: {'stop_lo

Best value found so far: 1068642.93424625


[I 2024-04-09 23:05:47,704] Trial 0 finished with value: 1059180.2600290638 and parameters: {'stop_loss': 0.8662597394914577, 'take_profit': 1.091855569961927, 'n_shares': 25}. Best is trial 0 with value: 1059180.2600290638.
[I 2024-04-09 23:05:48,166] Trial 1 finished with value: 1091498.7020536754 and parameters: {'stop_loss': 0.8750854417584945, 'take_profit': 1.0918780407851645, 'n_shares': 42}. Best is trial 1 with value: 1091498.7020536754.
[I 2024-04-09 23:05:48,311] Trial 2 finished with value: 998603.9928778111 and parameters: {'stop_loss': 0.8255923128399603, 'take_profit': 1.017303362836217, 'n_shares': 31}. Best is trial 1 with value: 1091498.7020536754.
[I 2024-04-09 23:05:48,783] Trial 3 finished with value: 1056757.6682662521 and parameters: {'stop_loss': 0.8718378299900466, 'take_profit': 1.088600966793718, 'n_shares': 25}. Best is trial 1 with value: 1091498.7020536754.
[I 2024-04-09 23:05:49,104] Trial 4 finished with value: 1046471.313968572 and parameters: {'stop_lo

Best value found so far: 1100582.3360849994


In [48]:
data_1h_test = pd.read_csv("data/aapl_1h_test.csv")
data_1h_test = data_1h_test.dropna()
dataresult_long_1h_test = file_features(data_1h_test, ds_type="buy")
dataresult_short_1h_test = file_features(data_1h_test, ds_type="sell")
global_buy_signals = buy_signals(dataresult_long_1h_test, logistic_params_1h_long, svm_params_1h_long, xgboost_params_1h_long)
global_sell_signals = sell_signals(dataresult_short_1h_test, logistic_params_1h_short, svm_params_1h_short, xgboost_params_1h_short)
file_1h_test = optimize_file(data_1h_test)

/opt/anaconda3/envs/trading/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2024-04-09 23:08:00,048] A new study created in memory with name: no-name-abdae2f9-6ed3-4b91-a382-980173f56357
[I 2024-04-09 23:08:04,952] Trial 0 finished with value: 606633.3591033877 and parameters: {'stop_loss': 0.8719892775971674, 'take_profit': 1.0310930403121354, 'n_shares': 24}. Best is trial 0 with value: 606633.3591033877.
[I 2024-04-09 23:08:15,555] Trial 1 finished with value: 1032469.3822081675 and parameters: {'stop_loss': 0.8460214683591737, 'take_profit': 1.0909900976483313, 'n_shares': 22}. Best is trial 1 with value: 1032469.3822081675.
[I 2024-04-09 23:08:18,553] Trial 2 finished with value: 321984.78305432183 and parameters: {'stop_loss': 0.876276624345988, 'take_profit': 1.0181434507282714, 'n_shares': 42}. Best is trial 1 with value: 1032469.3822081675.
[I 2024-04-09 23:08

Best value found so far: 1111710.9037487532
('logistic',)
{'stop_loss': 0.8165769544265113, 'take_profit': 1.0997042785411313, 'n_shares': 30}


[I 2024-04-09 23:10:11,957] Trial 0 finished with value: 971833.0293322356 and parameters: {'stop_loss': 0.802910187334884, 'take_profit': 1.0588097607647422, 'n_shares': 31}. Best is trial 0 with value: 971833.0293322356.
[I 2024-04-09 23:10:16,770] Trial 1 finished with value: 731098.5943805571 and parameters: {'stop_loss': 0.8400047980385039, 'take_profit': 1.0325992360587772, 'n_shares': 35}. Best is trial 0 with value: 971833.0293322356.
[I 2024-04-09 23:10:24,703] Trial 2 finished with value: 963669.4012156194 and parameters: {'stop_loss': 0.8344024324354309, 'take_profit': 1.0645503073742886, 'n_shares': 25}. Best is trial 0 with value: 971833.0293322356.
[I 2024-04-09 23:10:30,255] Trial 3 finished with value: 764212.0873123067 and parameters: {'stop_loss': 0.8927129442633976, 'take_profit': 1.0589699281512543, 'n_shares': 27}. Best is trial 0 with value: 971833.0293322356.
[I 2024-04-09 23:10:33,772] Trial 4 finished with value: 782113.9661392386 and parameters: {'stop_loss': 

Best value found so far: 1098970.8938993488


[I 2024-04-09 23:11:55,531] Trial 0 finished with value: 1054086.142974685 and parameters: {'stop_loss': 0.8063110779913605, 'take_profit': 1.054869944226592, 'n_shares': 25}. Best is trial 0 with value: 1054086.142974685.
[I 2024-04-09 23:11:58,391] Trial 1 finished with value: 878249.6504402168 and parameters: {'stop_loss': 0.895544921033185, 'take_profit': 1.0417089150104235, 'n_shares': 38}. Best is trial 0 with value: 1054086.142974685.
[I 2024-04-09 23:12:03,553] Trial 2 finished with value: 1001970.2804709298 and parameters: {'stop_loss': 0.87575335708729, 'take_profit': 1.0752234477533251, 'n_shares': 39}. Best is trial 0 with value: 1054086.142974685.
[I 2024-04-09 23:12:09,082] Trial 3 finished with value: 1041490.6943769562 and parameters: {'stop_loss': 0.8986236912529201, 'take_profit': 1.0867050194856591, 'n_shares': 28}. Best is trial 0 with value: 1054086.142974685.
[I 2024-04-09 23:12:16,146] Trial 4 finished with value: 1126675.8909622533 and parameters: {'stop_loss': 

...

Best value found so far: 1126590.3435831936


[I 2024-04-09 23:18:40,939] Trial 0 finished with value: 1077836.9022316884 and parameters: {'stop_loss': 0.820601297268971, 'take_profit': 1.0844646605441282, 'n_shares': 29}. Best is trial 0 with value: 1077836.9022316884.
[I 2024-04-09 23:18:50,668] Trial 1 finished with value: 904734.2231170613 and parameters: {'stop_loss': 0.8777361598591299, 'take_profit': 1.0953471602594727, 'n_shares': 22}. Best is trial 0 with value: 1077836.9022316884.
[I 2024-04-09 23:18:58,332] Trial 2 finished with value: 1058970.063502877 and parameters: {'stop_loss': 0.8031865544664679, 'take_profit': 1.0757448125979654, 'n_shares': 34}. Best is trial 0 with value: 1077836.9022316884.
[I 2024-04-09 23:19:03,216] Trial 3 finished with value: 723101.8555458739 and parameters: {'stop_loss': 0.8922012217420738, 'take_profit': 1.0856548998249156, 'n_shares': 46}. Best is trial 0 with value: 1077836.9022316884.
[I 2024-04-09 23:19:11,034] Trial 4 finished with value: 932649.1950929457 and parameters: {'stop_lo

Best value found so far: 1077836.9022316884


[I 2024-04-09 23:20:57,101] Trial 0 finished with value: 446319.70749503054 and parameters: {'stop_loss': 0.8216890728522166, 'take_profit': 1.0130647691483656, 'n_shares': 47}. Best is trial 0 with value: 446319.70749503054.
[I 2024-04-09 23:21:04,959] Trial 1 finished with value: 811724.3075696533 and parameters: {'stop_loss': 0.87469440338256, 'take_profit': 1.0782011187254426, 'n_shares': 28}. Best is trial 1 with value: 811724.3075696533.
[I 2024-04-09 23:21:09,895] Trial 2 finished with value: 835613.3867397825 and parameters: {'stop_loss': 0.8156163538827103, 'take_profit': 1.0407111332263548, 'n_shares': 47}. Best is trial 2 with value: 835613.3867397825.
[I 2024-04-09 23:21:15,001] Trial 3 finished with value: 764415.2436287573 and parameters: {'stop_loss': 0.8741062592135858, 'take_profit': 1.0771929077445423, 'n_shares': 45}. Best is trial 2 with value: 835613.3867397825.
[I 2024-04-09 23:21:22,223] Trial 4 finished with value: 769055.0181639832 and parameters: {'stop_loss':

Best value found so far: 1120338.5722740472


In [49]:
data_5m_test = pd.read_csv("data/aapl_5m_train.csv")
data_5m_test = data_5m_test.dropna()
dataresult_long_5m_test = file_features(data_5m_test, ds_type="buy")
dataresult_short_5m_test = file_features(data_5m_test, ds_type="sell")
global_buy_signals = buy_signals(dataresult_long_5m_test, logistic_params_5m_long, svm_params_5m_long, xgboost_params_5m_long)
global_sell_signals = sell_signals(dataresult_short_5m_test, logistic_params_5m_short, svm_params_5m_short, xgboost_params_5m_short)
file_5m_test = optimize_file(data_5m_test)

/opt/anaconda3/envs/trading/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2024-04-09 23:27:49,524] A new study created in memory with name: no-name-a731de12-dcce-4bf6-96a1-9510a770fe25
[I 2024-04-09 23:29:14,603] Trial 0 finished with value: 547301.9094562563 and parameters: {'stop_loss': 0.8406084283113947, 'take_profit': 1.0387133805659143, 'n_shares': 20}. Best is trial 0 with value: 547301.9094562563.
[I 2024-04-09 23:30:45,051] Trial 1 finished with value: 665255.9188125415 and parameters: {'stop_loss': 0.8093972484959612, 'take_profit': 1.0508878750841668, 'n_shares': 22}. Best is trial 1 with value: 665255.9188125415.
[I 2024-04-09 23:32:00,524] Trial 2 finished with value: 719315.6946571099 and parameters: {'stop_loss': 0.8521993679944919, 'take_profit': 1.0910832395763077, 'n_shares': 27}. Best is trial 2 with value: 719315.6946571099.
[I 2024-04-09 23:32:47

Best value found so far: 749646.4269762242
('logistic',)
{'stop_loss': 0.8039926867101456, 'take_profit': 1.0694255341609429, 'n_shares': 39}


[I 2024-04-09 23:46:52,920] Trial 0 finished with value: 719892.2816202762 and parameters: {'stop_loss': 0.8375766509856937, 'take_profit': 1.0796163863385868, 'n_shares': 43}. Best is trial 0 with value: 719892.2816202762.
[I 2024-04-09 23:47:38,066] Trial 1 finished with value: 465502.10806950607 and parameters: {'stop_loss': 0.8944618327555343, 'take_profit': 1.0604787125121664, 'n_shares': 40}. Best is trial 0 with value: 719892.2816202762.
[I 2024-04-09 23:48:27,416] Trial 2 finished with value: 333298.7014985611 and parameters: {'stop_loss': 0.8900248196881897, 'take_profit': 1.0396913418372817, 'n_shares': 35}. Best is trial 0 with value: 719892.2816202762.
[I 2024-04-09 23:49:38,844] Trial 3 finished with value: 579089.5755458128 and parameters: {'stop_loss': 0.8907317728489669, 'take_profit': 1.077470494177735, 'n_shares': 35}. Best is trial 0 with value: 719892.2816202762.
[I 2024-04-09 23:50:31,387] Trial 4 finished with value: 540405.2502055294 and parameters: {'stop_loss':

Best value found so far: 793729.9267015033
('svm',)
{'stop_loss': 0.8102634543726406, 'take_profit': 1.0858774446881216, 'n_shares': 30}


[I 2024-04-10 00:06:49,841] Trial 0 finished with value: 518319.36236740183 and parameters: {'stop_loss': 0.8693851966500652, 'take_profit': 1.0472567260527539, 'n_shares': 48}. Best is trial 0 with value: 518319.36236740183.
[I 2024-04-10 00:07:52,773] Trial 1 finished with value: 805803.5728252433 and parameters: {'stop_loss': 0.8262908706082416, 'take_profit': 1.0925415809495975, 'n_shares': 35}. Best is trial 1 with value: 805803.5728252433.
[I 2024-04-10 00:08:34,816] Trial 2 finished with value: 372717.98880237323 and parameters: {'stop_loss': 0.8221811001935114, 'take_profit': 1.01756485839336, 'n_shares': 35}. Best is trial 1 with value: 805803.5728252433.
[I 2024-04-10 00:09:15,877] Trial 3 finished with value: 654651.7171806254 and parameters: {'stop_loss': 0.8846887197418999, 'take_profit': 1.0853904079178538, 'n_shares': 50}. Best is trial 1 with value: 805803.5728252433.
[I 2024-04-10 00:10:07,897] Trial 4 finished with value: 643878.3102096333 and parameters: {'stop_loss'

...

Best value found so far: 817332.544994896


[I 2024-04-10 01:06:28,740] Trial 0 finished with value: 463105.33396201435 and parameters: {'stop_loss': 0.8612548089124559, 'take_profit': 1.041042834174395, 'n_shares': 40}. Best is trial 0 with value: 463105.33396201435.
[I 2024-04-10 01:07:41,737] Trial 1 finished with value: 434173.3400979335 and parameters: {'stop_loss': 0.8519540181423324, 'take_profit': 1.0310451790959336, 'n_shares': 22}. Best is trial 0 with value: 463105.33396201435.
[I 2024-04-10 01:08:24,752] Trial 2 finished with value: 175320.24228617302 and parameters: {'stop_loss': 0.8403022662530488, 'take_profit': 1.0118244663810236, 'n_shares': 26}. Best is trial 0 with value: 463105.33396201435.
[I 2024-04-10 01:09:36,106] Trial 3 finished with value: 817131.0909547779 and parameters: {'stop_loss': 0.8050862189335702, 'take_profit': 1.091662907232504, 'n_shares': 31}. Best is trial 3 with value: 817131.0909547779.
[I 2024-04-10 01:11:05,416] Trial 4 finished with value: 503554.37823988637 and parameters: {'stop_lo

Best value found so far: 817423.9783429252


[I 2024-04-10 01:24:51,848] Trial 0 finished with value: 541227.7961859823 and parameters: {'stop_loss': 0.8123014107627383, 'take_profit': 1.0278496800983339, 'n_shares': 24}. Best is trial 0 with value: 541227.7961859823.
[I 2024-04-10 01:25:31,913] Trial 1 finished with value: 458094.2639513999 and parameters: {'stop_loss': 0.8650446472813839, 'take_profit': 1.042553814004602, 'n_shares': 44}. Best is trial 0 with value: 541227.7961859823.
[I 2024-04-10 01:26:12,423] Trial 2 finished with value: 583299.2209496973 and parameters: {'stop_loss': 0.8282921759133368, 'take_profit': 1.044809688577803, 'n_shares': 46}. Best is trial 2 with value: 583299.2209496973.
[I 2024-04-10 01:27:25,276] Trial 3 finished with value: 536150.8660680465 and parameters: {'stop_loss': 0.8959357806277274, 'take_profit': 1.0725446131545167, 'n_shares': 26}. Best is trial 2 with value: 583299.2209496973.
[I 2024-04-10 01:28:29,601] Trial 4 finished with value: 615449.4667109519 and parameters: {'stop_loss': 0

Best value found so far: 737426.0761746248
